In [1]:
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from functools import reduce

from darts import TimeSeries
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    Prophet,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RegressionEnsembleModel,
    RegressionModel,
    Theta,
    FFT,
    RNNModel
)

from darts.metrics import mape, mase, mae, rmse
from darts.utils.statistics import check_seasonality, plot_acf, plot_residuals_analysis
from darts.datasets import AirPassengersDataset

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
import pickle

# Delta 1

In [43]:
name = 'GOOG'

In [44]:
delta = 1

In [45]:
df = pd.read_csv("Stocks.csv", parse_dates=["Date"], index_col="Date")

In [46]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2018-01-02,43.0650,177.7000,1189.0100,181.4200,1065.0000,46.6300
2018-01-03,43.0575,181.0400,1204.2000,184.6700,1082.4800,47.7100
2018-01-04,43.2575,183.2200,1209.5900,184.3300,1086.4000,48.1800
2018-01-05,43.7500,185.3400,1229.1400,186.8500,1102.2300,48.4700
2018-01-06,43.6958,185.2400,1235.0500,187.3267,1103.8000,48.6400
...,...,...,...,...,...,...
2021-05-30,124.4450,500.2450,3220.8600,328.9300,2420.6851,79.1550
2021-05-31,124.3625,498.0775,3219.7549,329.0300,2425.2476,79.3625
2021-06-01,124.2800,495.9100,3218.6499,329.1300,2429.8101,79.5700


In [47]:
df = df.loc[pd.Timestamp("20200101"):pd.Timestamp("20210101"), :]

In [48]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2020-01-01,74.2500,332.1200,1872.9250,207.515,1352.1950,53.4650
2020-01-02,75.0875,334.4300,1898.0100,209.780,1367.3700,53.9500
2020-01-03,74.3575,331.8100,1874.9700,208.670,1360.6600,53.7600
2020-01-04,74.5550,332.4433,1884.2733,209.980,1371.8433,53.8533
2020-01-05,74.7525,333.0767,1893.5767,211.290,1383.0267,53.9467
...,...,...,...,...,...,...
2020-12-28,136.6900,498.9500,3283.9600,277.000,1776.0900,64.8700
2020-12-29,134.8700,502.1100,3322.0000,276.780,1758.7200,64.4600
2020-12-30,133.7200,497.4500,3285.8501,271.870,1739.5200,64.4000


In [49]:
series = TimeSeries.from_series(df[name])

## Darts

In [50]:
models = [ExponentialSmoothing(), AutoARIMA(), Prophet()]

In [51]:
import functools

backtests = []

for model in models:
    print(f"{model}: running...")
    initial_time = time.time()
    hist_pred = model.historical_forecasts(series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
    hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
    backtests.append(hist_pred)
    
    final_time = time.time() - initial_time
    print(f"{model}: final time spent: {round(final_time, 3)}")

  0%|          | 0/93 [00:00<?, ?it/s]

Exponential smoothing: 15.647887468338013


  0%|          | 0/93 [00:00<?, ?it/s]

Auto-ARIMA: 201.98820567131042


  0%|          | 0/93 [00:00<?, ?it/s]

Prophet: 146.53497457504272


In [61]:
from darts.dataprocessing.transformers import Scaler

print(f"LSTM: running...")
initial_time = time.time()
transformer = Scaler()
transformed_series = transformer.fit_transform(series)
lstm = RNNModel(model='LSTM', input_chunk_length=round(len(series)/4), output_chunk_length=1)
models.append(lstm)

hist_pred =  lstm.historical_forecasts(transformed_series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
backtests.append(transformer.inverse_transform(hist_pred))
final_time = time.time() - initial_time
print(f"LSTM: final time spent: {round(final_time, 3)}")

  0%|          | 0/93 [00:00<?, ?it/s]

In [68]:
darts_maes = {}
darts_rmses = {}

for i, m in enumerate(models):
    prediction = backtests[i]
    err_mae = mae(backtests[i], series)
    err_rmse = rmse(backtests[i], series)
    darts_maes[m] = err_mae
    darts_rmses[m] = err_rmse

Exponential smoothing: MAE = 17.447, RMSE = 24.038
Auto-ARIMA: MAE = 15.62, RMSE = 22.656
Prophet: MAE = 77.055, RMSE = 91.603
<darts.models.rnn_model.RNNModel object at 0x7f7d41831f10>: MAE = 20.132, RMSE = 28.533


IndexError: list index out of range

## Timex

In [69]:
with open(f"univariate/delta_{delta}/historical_predictions.pkl", 'rb') as file:
    p = pickle.load(file)

In [70]:
timex_maes = {}
timex_rmses = {}

for i, m in enumerate(p):
    pred_timex = p[m]
    pred_timex = pred_timex[name].astype('float')
    pred_timex = TimeSeries.from_series(pred_timex)
    pred_timex = pred_timex.slice_intersect(backtests[0])
    #     pred_timex = pred_timex.drop_after(backtests[0].time_index()[-1] + pd.Timedelta(days=1))
    assert len(pred_timex) == len(backtests[i])

    err_mae = mae(pred_timex, series)
    err_rmse = rmse(pred_timex, series)

    timex_maes[m] = err_mae
    timex_rmses[m] = err_rmse

In [71]:
print("########## FINAL RESULTS ##########")
print(f"Case: GOOG, delta: {delta}")
print("MAES")
print("Darts results:")
for i, m in enumerate(darts_maes):
    print(f"{m}, MAE={round(darts_maes[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_maes):
    print(f"{m}, MAE={round(timex_maes[m], 3)}")

print("------------------------")
print("RMSES")
print("Darts results:")

for i, m in enumerate(darts_rmses):
    print(f"{m}, RMSE={round(darts_rmses[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_rmses):
    print(f"{m}, RMSE={round(timex_rmses[m], 3)}")

MAES
Darts
Exponential smoothing, MAE=17.44748591123813
Auto-ARIMA, MAE=15.620277922686217
Prophet, MAE=77.0546584791413
<darts.models.rnn_model.RNNModel object at 0x7f7d41831f10>, MAE=20.131912993531586
Timex
fbprophet, MAE=38.21203164709654
arima, MAE=15.164720199091901
exponentialsmoothing, MAE=16.532371283289685
lstm, MAE=27.634364585690204
------------------------
RMSES
Darts
Exponential smoothing, RMSE=24.03809959751226
Auto-ARIMA, RMSE=22.656364103013303
Prophet, RMSE=91.60279417481226
<darts.models.rnn_model.RNNModel object at 0x7f7d41831f10>, RMSE=28.53290194329017
Timex
fbprophet, RMSE=46.763024975392575
arima, RMSE=22.41837875809258
exponentialsmoothing, RMSE=23.48546741518644
lstm, RMSE=36.722015767905184


# Delta 3

In [72]:
delta = 3

In [73]:
df = pd.read_csv("Stocks.csv", parse_dates=["Date"], index_col="Date")

In [74]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2018-01-02,43.0650,177.7000,1189.0100,181.4200,1065.0000,46.6300
2018-01-03,43.0575,181.0400,1204.2000,184.6700,1082.4800,47.7100
2018-01-04,43.2575,183.2200,1209.5900,184.3300,1086.4000,48.1800
2018-01-05,43.7500,185.3400,1229.1400,186.8500,1102.2300,48.4700
2018-01-06,43.6958,185.2400,1235.0500,187.3267,1103.8000,48.6400
...,...,...,...,...,...,...
2021-05-30,124.4450,500.2450,3220.8600,328.9300,2420.6851,79.1550
2021-05-31,124.3625,498.0775,3219.7549,329.0300,2425.2476,79.3625
2021-06-01,124.2800,495.9100,3218.6499,329.1300,2429.8101,79.5700


In [75]:
df = df.loc[pd.Timestamp("20200101"):pd.Timestamp("20210101"), :]

In [76]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2020-01-01,74.2500,332.1200,1872.9250,207.515,1352.1950,53.4650
2020-01-02,75.0875,334.4300,1898.0100,209.780,1367.3700,53.9500
2020-01-03,74.3575,331.8100,1874.9700,208.670,1360.6600,53.7600
2020-01-04,74.5550,332.4433,1884.2733,209.980,1371.8433,53.8533
2020-01-05,74.7525,333.0767,1893.5767,211.290,1383.0267,53.9467
...,...,...,...,...,...,...
2020-12-28,136.6900,498.9500,3283.9600,277.000,1776.0900,64.8700
2020-12-29,134.8700,502.1100,3322.0000,276.780,1758.7200,64.4600
2020-12-30,133.7200,497.4500,3285.8501,271.870,1739.5200,64.4000


In [77]:
series = TimeSeries.from_series(df[name])

## Darts

In [78]:
models = [ExponentialSmoothing(), AutoARIMA(), Prophet()]

In [79]:
import functools

backtests = []

for model in models:
    print(f"{model}: running...")
    initial_time = time.time()
    hist_pred = model.historical_forecasts(series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
    hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
    backtests.append(hist_pred)
    
    final_time = time.time() - initial_time
    print(f"{model}: final time spent: {round(final_time, 3)}")

  0%|          | 0/31 [00:00<?, ?it/s]

Exponential smoothing: 5.40540623664856


  0%|          | 0/31 [00:00<?, ?it/s]

Auto-ARIMA: 69.64171171188354


  0%|          | 0/31 [00:00<?, ?it/s]

Prophet: 50.172332525253296


In [80]:
from darts.dataprocessing.transformers import Scaler

print(f"LSTM: running...")
initial_time = time.time()
transformer = Scaler()
transformed_series = transformer.fit_transform(series)
lstm = RNNModel(model='LSTM', input_chunk_length=round(len(series)/4), output_chunk_length=1)
models.append(lstm)

hist_pred =  lstm.historical_forecasts(transformed_series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
backtests.append(transformer.inverse_transform(hist_pred))
final_time = time.time() - initial_time
print(f"LSTM: final time spent: {round(final_time, 3)}")

  0%|          | 0/31 [00:00<?, ?it/s]

In [81]:
darts_maes = {}
darts_rmses = {}

for i, m in enumerate(models):
    prediction = backtests[i]
#     print(prediction)
    err_mae = mae(backtests[i], series)
    err_rmse = rmse(backtests[i], series)
    darts_maes[m] = err_mae
    darts_rmses[m] = err_rmse
    
#     print(f"{m}: MAE = {round(err_mae, 3)}, RMSE = {round(err_rmse, 3)}")

## Timex

In [82]:
with open(f"univariate/delta_{delta}/historical_predictions.pkl", 'rb') as file:
    p = pickle.load(file)

In [83]:
timex_maes = {}
timex_rmses = {}

for i, m in enumerate(p):
    pred_timex = p[m]
    pred_timex = pred_timex[name].astype('float')
    pred_timex = TimeSeries.from_series(pred_timex)
    pred_timex = pred_timex.slice_intersect(backtests[0])
#     pred_timex = pred_timex.drop_after(backtests[0].time_index()[-1] + pd.Timedelta(days=1))
    assert len(pred_timex) == len(backtests[0])
    
    err_mae = mae(pred_timex, series)
    err_rmse = rmse(pred_timex, series)
    
    timex_maes[m] = err_mae
    timex_rmses[m] = err_rmse

In [84]:
print("########## FINAL RESULTS ##########")
print(f"Case: GOOG, delta: {delta}")
print("MAES")
print("Darts results:")
for i, m in enumerate(darts_maes):
    print(f"{m}, MAE={round(darts_maes[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_maes):
    print(f"{m}, MAE={round(timex_maes[m], 3)}")

print("------------------------")
print("RMSES")
print("Darts results:")

for i, m in enumerate(darts_rmses):
    print(f"{m}, RMSE={round(darts_rmses[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_rmses):
    print(f"{m}, RMSE={round(timex_rmses[m], 3)}")

MAES
Darts
Exponential smoothing, MAE=23.40813658028029
Auto-ARIMA, MAE=20.439006820363193
Prophet, MAE=81.25926128058127
<darts.models.rnn_model.RNNModel object at 0x7f7d2afa1f70>, MAE=33.51365038642473
Timex
fbprophet, MAE=47.300698063447584
arima, MAE=22.154747304317564
exponentialsmoothing, MAE=20.20913535397744
lstm, MAE=35.30328940368547
------------------------
RMSES
Darts
Exponential smoothing, RMSE=34.91917806475344
Auto-ARIMA, RMSE=30.553150180593107
Prophet, RMSE=96.06817649066805
<darts.models.rnn_model.RNNModel object at 0x7f7d2afa1f70>, RMSE=46.470334590220666
Timex
fbprophet, RMSE=56.98556734790414
arima, RMSE=32.55308625236377
exponentialsmoothing, RMSE=30.88810564388966
lstm, RMSE=47.88170385921885


# Delta 7


In [85]:
delta = 7

In [86]:
df = pd.read_csv("Stocks.csv", parse_dates=["Date"], index_col="Date")

In [87]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2018-01-02,43.0650,177.7000,1189.0100,181.4200,1065.0000,46.6300
2018-01-03,43.0575,181.0400,1204.2000,184.6700,1082.4800,47.7100
2018-01-04,43.2575,183.2200,1209.5900,184.3300,1086.4000,48.1800
2018-01-05,43.7500,185.3400,1229.1400,186.8500,1102.2300,48.4700
2018-01-06,43.6958,185.2400,1235.0500,187.3267,1103.8000,48.6400
...,...,...,...,...,...,...
2021-05-30,124.4450,500.2450,3220.8600,328.9300,2420.6851,79.1550
2021-05-31,124.3625,498.0775,3219.7549,329.0300,2425.2476,79.3625
2021-06-01,124.2800,495.9100,3218.6499,329.1300,2429.8101,79.5700


In [88]:
df = df.loc[pd.Timestamp("20200101"):pd.Timestamp("20210101"), :]

In [89]:
df

,AAPL,ADBE,AMZN,FB,GOOG,ORCL
Date,,,,,,
2020-01-01,74.2500,332.1200,1872.9250,207.515,1352.1950,53.4650
2020-01-02,75.0875,334.4300,1898.0100,209.780,1367.3700,53.9500
2020-01-03,74.3575,331.8100,1874.9700,208.670,1360.6600,53.7600
2020-01-04,74.5550,332.4433,1884.2733,209.980,1371.8433,53.8533
2020-01-05,74.7525,333.0767,1893.5767,211.290,1383.0267,53.9467
...,...,...,...,...,...,...
2020-12-28,136.6900,498.9500,3283.9600,277.000,1776.0900,64.8700
2020-12-29,134.8700,502.1100,3322.0000,276.780,1758.7200,64.4600
2020-12-30,133.7200,497.4500,3285.8501,271.870,1739.5200,64.4000


In [90]:
series = TimeSeries.from_series(df[name])

## Darts

In [91]:
models = [ExponentialSmoothing(), AutoARIMA(), Prophet()]

In [92]:
import functools

backtests = []

for model in models:
    print(f"{model}: running...")
    initial_time = time.time()
    hist_pred = model.historical_forecasts(series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
    hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
    backtests.append(hist_pred)
    
    final_time = time.time() - initial_time
    print(f"{model}: final time spent: {round(final_time, 3)}")

  0%|          | 0/13 [00:00<?, ?it/s]

Exponential smoothing: 2.40457820892334


  0%|          | 0/13 [00:00<?, ?it/s]

Auto-ARIMA: 31.317824602127075


  0%|          | 0/13 [00:00<?, ?it/s]

Prophet: 22.12063217163086


In [93]:
from darts.dataprocessing.transformers import Scaler

print(f"LSTM: running...")
initial_time = time.time()
transformer = Scaler()
transformed_series = transformer.fit_transform(series)
lstm = RNNModel(model='LSTM', input_chunk_length=round(len(series)/4), output_chunk_length=1)
models.append(lstm)

hist_pred =  lstm.historical_forecasts(transformed_series,
                            start=pd.Timestamp('2020-10-01'),
                            forecast_horizon=delta, stride=delta, verbose=True, last_points_only=False)
hist_pred = functools.reduce(lambda a, b: a.append(b), hist_pred)
backtests.append(transformer.inverse_transform(hist_pred))
final_time = time.time() - initial_time
print(f"LSTM: final time spent: {round(final_time, 3)}")

  0%|          | 0/13 [00:00<?, ?it/s]

In [94]:
darts_maes = {}
darts_rmses = {}

for i, m in enumerate(models):
    prediction = backtests[i]
    #     print(prediction)
    err_mae = mae(backtests[i], series)
    err_rmse = rmse(backtests[i], series)
    darts_maes[m] = err_mae
    darts_rmses[m] = err_rmse


Exponential smoothing: MAE = 34.288, RMSE = 50.291
Auto-ARIMA: MAE = 31.225, RMSE = 46.524
Prophet: MAE = 89.585, RMSE = 105.583
<darts.models.rnn_model.RNNModel object at 0x7f7d41acc070>: MAE = 34.939, RMSE = 52.087


## Timex

In [95]:
with open(f"univariate/delta_{delta}/historical_predictions.pkl", 'rb') as file:
    p = pickle.load(file)

In [96]:
timex_maes = {}
timex_rmses = {}

for i, m in enumerate(p):
    pred_timex = p[m]
    pred_timex = pred_timex[name].astype('float')
    pred_timex = TimeSeries.from_series(pred_timex)
    pred_timex = pred_timex.slice_intersect(backtests[0])
    #     pred_timex = pred_timex.drop_after(backtests[0].time_index()[-1] + pd.Timedelta(days=1))
    assert len(pred_timex) == len(backtests[0])

    err_mae = mae(pred_timex, series)
    err_rmse = rmse(pred_timex, series)

    timex_maes[m] = err_mae
    timex_rmses[m] = err_rmse

In [97]:
print("########## FINAL RESULTS ##########")
print(f"Case: GOOG, delta: {delta}")
print("MAES")
print("Darts results:")
for i, m in enumerate(darts_maes):
    print(f"{m}, MAE={round(darts_maes[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_maes):
    print(f"{m}, MAE={round(timex_maes[m], 3)}")

print("------------------------")
print("RMSES")
print("Darts results:")

for i, m in enumerate(darts_rmses):
    print(f"{m}, RMSE={round(darts_rmses[m], 3)}")

print("Timex results:")
for i, m in enumerate(timex_rmses):
    print(f"{m}, RMSE={round(timex_rmses[m], 3)}")

MAES
Darts
Exponential smoothing, MAE=34.28782247460107
Auto-ARIMA, MAE=31.22474167515422
Prophet, MAE=89.58500705974191
<darts.models.rnn_model.RNNModel object at 0x7f7d41acc070>, MAE=34.93880478193681
Timex
arima, MAE=30.973379410338936
exponentialsmoothing, MAE=33.930088311320276
fbprophet, MAE=58.72355944556571
lstm, MAE=52.514072561406444
------------------------
RMSES
Darts
Exponential smoothing, RMSE=50.291091892634235
Auto-ARIMA, RMSE=46.523861204948425
Prophet, RMSE=105.5830678691845
<darts.models.rnn_model.RNNModel object at 0x7f7d41acc070>, RMSE=52.086712668960395
Timex
arima, RMSE=45.34777113063382
exponentialsmoothing, RMSE=49.60169328954722
fbprophet, RMSE=69.3634804303133
lstm, RMSE=64.08618607517573
